Import Necessary packages

In [ ]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
import matplotlib.pyplot as plt
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.preprocessing.image import load_img, img_to_array
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

Locate Path and different Parameters

In [ ]:
# Set some parameters
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
TRAIN_PATH = 'train'
TEST_PATH = 'test'
TRAIN_MASK_PATH = 'train_masks'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

Read Test images and collect

In [ ]:
n = len(os.listdir(TRAIN_PATH))
X = np.zeros((n, IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8)
p=0
for label in sorted(os.listdir(TRAIN_PATH)):
    path = os.path.join(TRAIN_PATH,label)
    img = load_img(path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img = img_to_array(img)
    X[p] = img
    p = p+1

print(X.shape)

Show one of the Test Image

In [ ]:
plt.imshow(X[1])

Read the corresponding mask images and store it

In [ ]:
Y = np.zeros((n, IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.uint8)
p = 0
for label in sorted(os.listdir(TRAIN_MASK_PATH)):
    path = os.path.join(TRAIN_MASK_PATH, label)
    img = load_img(path, target_size=(IMG_HEIGHT, IMG_WIDTH), grayscale=True)
    img = img_to_array(img)
    Y[p] = img
    p = p + 1

Y = Y == 255
print(Y.shape)

plt.imshow(np.squeeze(Y[1]))

Collect 100 data for test set

In [ ]:
X_test = np.zeros((100, IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8)
p=0
for label in sorted(os.listdir(TEST_PATH)):
    path = os.path.join(TEST_PATH,label)
    img = load_img(path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img = img_to_array(img)
    X_test[p] = img
    p = p+1
    if p==100:
        break

print(X_test.shape)

Define IoU metric

In [ ]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        #K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    #return K.mean(K.stack(prec), axis=0)

Build U-Net model

In [ ]:
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (s)
c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (p4)
c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (c5)

u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
model.summary()

Fit model

In [ ]:
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
results = model.fit(X, Y, validation_split=0.1, batch_size=32, epochs=20,
                    callbacks=[earlystopper, checkpointer])


Predict on train, val and test

In [ ]:
model = load_model('model-dsbowl2018-1.h5', custom_objects={'mean_iou': mean_iou})
preds_train = model.predict(X[:int(X.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X[int(X.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

Threshold predictions

In [ ]:
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

Convert 0 and 1 to 0 and 255 to distinguish mask from background

In [ ]:
preds_train_t[preds_train_t==1] =  255
preds_val_t[preds_val_t==1] = 255
preds_test_t[preds_test_t==1] = 255

 Squeeze for plot

In [ ]:
preds_train_t = np.squeeze(preds_train_t, axis=-1)
preds_val_t = np.squeeze(preds_val_t, axis=-1)
preds_test_t = np.squeeze(preds_test_t, axis=-1)


Pot a test car image, its correxponding mask and the masked car

In [ ]:
import cv2
img = X_test[80]
mask = preds_test_t[80]
mask_img = cv2.bitwise_and(img, img, mask=mask)
plt.subplot(131)
plt.imshow(img)
plt.subplot(132)
plt.imshow(mask)
plt.subplot(133)
plt.imshow(mask_img)